<a href="https://colab.research.google.com/github/thefr33radical/courses/blob/master/nlp/materials/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:


from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#!`--NotebookApp.iopub_data_rate_limit`.
#!pip3 install bs4
#!pip3 install gensim

/bin/bash: --NotebookApp.iopub_data_rate_limit: command not found
/bin/bash: line 0: .: filename argument required
.: usage: . filename [arguments]


In [0]:
from bs4 import BeautifulSoup
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import stem_text
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import strip_non_alphanum

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from scipy.sparse import csr_matrix
from sklearn.decomposition import PCA

import gensim
import os
import pandas as pd
import csv
import re
import math

class Salient(object):  
  
  def generate_data(self,soup):
    sent=soup.find_all("context")
    for i in sent:      
      yield i   
  
  def collocation(self,context_str):
    collc=[]
    cvocab=[]
    
    with open(path+"wsd_data.xml","rb") as f:
      data=f.read()

      soup=BeautifulSoup(data,'html.parser') 
      count =0     
      c=1
      count=0
      head_str=[]
      for s in (self.generate_data(soup)):
        try:
          
          head_str=re.findall("<head>(.*?)</head>", str(s))
          if len(head_str)<1 or head_str[0]=="":
            continue
          
          temp=s.text
          if temp=="":
            continue

          temp=strip_non_alphanum(temp)
          #temp =remove_stopwords(temp)
          temp=str.lower(temp)
          #temp =stem_text(temp)

          sent_list= temp.split(" ")          

          index=sent_list.index(head_str[0])
                  
          v_sent= sent_list[index-1:index+1]
          
          for v in v_sent:   
            
            if v not in cvocab and len(v)>3:
              cvocab.append(c) 
              c+=1

          head_str=[]
        except:
          pass
    print(cvocab)    
    with open(path+"cvocabs.csv","w+") as f:
      wr=csv.DictWriter(f,["cvocab"])
      
      for i in cvocab:
        wr.writerow({'cvocab': i})   
      
  
  def ReadData(self,path):
    """
    Function to read data and form vocabulary
    """
    docs=[]
    vocab=[]
    
    with open(path+"wsd_data.xml","rb") as f:
      data=f.read()

      soup=BeautifulSoup(data,'html.parser') 
      count =0     

      count=0
      head_str=[]
      for s in (self.generate_data(soup)):
        try:
          
          head_str=re.findall("<head>(.*?)</head>", str(s))
          if len(head_str)<1 or head_str[0]=="":
            continue
          
          temp=s.text
          if temp=="":
            continue

          temp=strip_non_alphanum(temp)
          #temp =remove_stopwords(temp)
          temp=str.lower(temp)
          #temp =stem_text(temp)

          sent_list= temp.split(" ")          

          index=sent_list.index(head_str[0])
                  
          v_sent= sent_list[index-1:index+1]
          for v in v_sent:            
            if v not in vocab and len(v)>3:
              vocab.append(v)          

          docs.append(temp)
          head_str=[]
        except:
          pass
    
    with open(path+"docs.csv","w+") as f:
      wr=csv.DictWriter(f,["document_name"])
      for i in docs:
        wr.writerow({'document_name': i})
        
    with open(path+"vocabs.csv","w+") as f:
      wr=csv.DictWriter(f,["vocab"])
      for i in vocab:
        wr.writerow({'vocab': i}) 
        
   
  def TF(self,path):
    """
    Function to compute Term Frequency matrix
    
    """
    docs=pd.read_csv(path+"docs.csv")
    docs.columns=["_DOCS"]
    docs.dropna(subset=['_DOCS'], how='any', inplace = True)

    #print(docs)
    
    v=pd.read_csv(path+"vocabs.csv")
    vocab=list(v.iloc[:,0])
    vocab=vocab[:500]
    TF=pd.DataFrame()
    for i in vocab[:500]:
      TF[i]=[0]*len(docs)
      
    TF["_DOCS"]=list(docs["_DOCS"])
    #TF=TF.dropna(subset=['_DOCS'], how='any', inplace = True)
    #print(TF)
   
    total_docs=len(docs)
    print(total_docs,len(TF.columns))
    
    
    for word in vocab:
      count=0
      total_docs=0
      
      for d in range(len(TF["_DOCS"])):
             
        try:
          
          count=TF.loc[d,"_DOCS"].count(word )
          total_len=len(TF.loc[d,"_DOCS"])
          TF.loc[d,word]=float(count/total_len)
          if count>0:
            print(stem_text(word), stem_text(TF.loc[d,"_DOCS"]))
            print(TF.loc[d,word],word,TF.loc[d,"_DOCS"])          
        except Exception as e:
          print(word,TF.loc[d,"_DOCS"],e)
          pass
        
    TF.to_csv(path+"TF.csv")
      
      
  def IDF(self,path):
    docs=pd.read_csv(path+"docs.csv")
    v=pd.read_csv(path+"vocabs.csv")
    vc=v.iloc[:,0]
    vectorizer = TfidfVectorizer(vocabulary=vc)
    X = vectorizer.fit_transform(docs.iloc[:,0])
    print(X.shape)
    newX=X.toarray()
    new_data=pd.DataFrame(newX)    
    new_data.to_csv(path+"IDF.csv")
    
    pca = PCA(n_components=10)
    result=pca.fit_transform(newX) 
    print(pca.explained_variance_ratio_)
    
    
obj=Salient()

path='/content/drive/My Drive/'
#docs=obj.ReadData('/content/drive/My Drive/')
#obj.collocation(path)
#obj.TF(path)
obj.IDF(path)